In [1]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import svm
import xgboost as xgb
import random
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [2]:
data=np.array(pd.read_csv('../user_data/big_train.csv').iloc[:,1:].drop('50',axis = 1))
X=data[:,:134]
y=data[:,134]



X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.00001) #划分训练集和测试集


In [3]:
#创建模型 nums:模型个数
from concurrent.futures import thread
from zmq import device


def make_models(nums,max_depth_list,n_estimators_list,learning_rate_list):
    models=[]
    for i in range(nums):
        max_depth=random.choice(max_depth_list)
        n_estimators=random.choice(n_estimators_list)
        learning_rate=random.choice(learning_rate_list)
        models.append((str(i),xgb.XGBClassifier(max_depth=max_depth,learning_rate=learning_rate,subsample=1, min_child_weight = 1, gamma=0.2,reg_alpha=0.09999999999999999,
         n_estimators=n_estimators,nthread = 4 ,scale_pos_weight=0.17660901285038)))
    return models 

In [4]:


models=make_models(512,list(range(3,7,1)),list(range(100,175,2)),[0.1,0.05,0.01,0.19,0.15,0.02,0.03,0.04,0.12,0.17])
meta_model = LogisticRegression()

In [5]:
stacking_model = StackingClassifier(estimators=models,cv=5, n_jobs=4)
#stacking_model = VotingClassifier(estimators=models)

In [6]:
stacking_model.fit(X_train, y_train)

In [ ]:
score_pre = f1_score(y_test,stacking_model.predict(X_test),average='macro') #计算F1-score
print(score_pre) 

1.0


In [ ]:
XX=np.array(pd.read_csv('../user_data/big_test.csv').iloc[:,1:].drop('50',axis = 1))

a=stacking_model.predict(XX) #预测
#保存预测的结果
pd.DataFrame({'id':pd.read_csv('../user_data/big_test.csv').iloc[:,0],
              'target':a}).to_csv('../prediction_result/result.csv',index=False)